In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf 
from PIL import Image
import os 
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

data = []
labels = []
classes = 43 
cur_path = os.getcwd() 

for i in range(classes): 
    path = os. path.join(cur_path,'./road/Train', str(i)) 
    images = os.listdir(path) 
    for a in images: 
        try: 
            image = Image.open(path + '/'+ a) 
            image = image.resize((30,30)) 
            image = np.array(image) 
            data.append(image) 
            labels.append(i) 
        except: 
            print("Error loading image") 
data = np.array(data)
labels = np.array(labels)
len(data)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_31520\359670903.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data)


ValueError: could not broadcast input array from shape (30,30,3) into shape (30,30)

In [5]:
print(data.shape, labels.shape)
#Splitting training and testing dataset
X_t1, X_t2, y_t1, y_t2 = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_t1.shape, X_t2.shape, y_t1.shape, y_t2.shape)
#Converting the labels into one hot encoding
y_t1 = to_categorical(y_t1, 43)
y_t2 = to_categorical(y_t2, 43)

(39209, 30, 30, 3) (39209,)
(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


In [7]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_t1.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))
#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
eps = 15
anc = model.fit(X_t1, y_t1, batch_size=32, epochs=eps, validation_data=(X_t2, y_t2))

Epoch 1/15
981/981 [==============================] - 46s 46ms/step - loss: 2.1084 - accuracy: 0.4679 - val_loss: 0.6048 - val_accuracy: 0.8590
Epoch 2/15
981/981 [==============================] - 52s 53ms/step - loss: 0.8487 - accuracy: 0.7557 - val_loss: 0.3369 - val_accuracy: 0.9100
Epoch 3/15
981/981 [==============================] - 51s 52ms/step - loss: 0.6201 - accuracy: 0.8189 - val_loss: 0.2432 - val_accuracy: 0.9236
Epoch 4/15
981/981 [==============================] - 51s 52ms/step - loss: 0.5340 - accuracy: 0.8440 - val_loss: 0.2065 - val_accuracy: 0.9393
Epoch 5/15
981/981 [==============================] - 51s 52ms/step - loss: 0.4452 - accuracy: 0.8673 - val_loss: 0.1594 - val_accuracy: 0.9513
Epoch 6/15
981/981 [==============================] - 51s 52ms/step - loss: 0.4121 - accuracy: 0.8769 - val_loss: 0.1202 - val_accuracy: 0.9666
Epoch 7/15
981/981 [==============================] - 55s 56ms/step - loss: 0.3696 - accuracy: 0.8899 - val_loss: 0.1026 - val_accuracy:

In [17]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('./road/Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open('./road/' + img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test = np.array(data)

In [20]:
pred = model.predict(X_test)
#Accuracy with the test data
print(pred)
# from sklearn.metrics import accuracy_score
# print(accuracy_score(labels, pred))
model.save('traffic_classifier.h5')

395/395 [==============================] - 3s 8ms/step
[[1.5049143e-25 6.4249314e-18 2.8344870e-18 ... 3.7354841e-10
  3.8872060e-19 1.9691699e-23]
 [9.0390949e-13 9.9999321e-01 4.8300734e-07 ... 3.5798769e-19
  1.0983592e-34 7.1054115e-33]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [1.6240034e-05 2.0507220e-02 2.2116829e-01 ... 1.6110933e-05
  2.9747187e-05 2.5785975e-03]
 [3.3064992e-10 1.8825796e-05 2.4849362e-07 ... 1.0781520e-06
  3.2895727e-13 5.7781538e-08]
 [1.2862567e-09 1.3224446e-06 6.7557266e-05 ... 3.4864926e-05
  1.6458348e-07 1.8015459e-06]]


In [21]:
model.save('traffic_classifier.h5')